# Analyse area source density 

In [2]:
%matplotlib inline
import re
import os
import sys
import glob
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

sys.path.append('/Users/mpagani/Repos/original/oq-engine/')
sys.path.append('/Users/mpagani/Repos/oq-man/')
sys.path.append('/Users/mpagani/Repos/model_building_tools/')

from openquake.hazardlib.const import TRT

import oqman.man.checking_utils.source_model_utils as model
from oqman.man.single_source_utils.areas_utils import get_rates_density
from openquake.mbt.mfd_tools import EEvenlyDiscretizedMFD, get_evenlyDiscretizedMFD_from_truncatedGRMFD
from openquake.hazardlib.mfd import TruncatedGRMFD

ModuleNotFoundError: No module named 'matplotlib'

## List of models to be analysed

In [ ]:
path = './pkl/*.pkl'
path = '/Users/mpagani/NC/Hazard_Charles/Documents/Presentations/2017/0417_SSA/notebooks/pkl/*.pkl'
modell = set(['als07', 'aus12', 'ca10h', 'ca10r', 'cub03', 'eu13a', 'eu13f',
              'eu13s', 'ear16', 'em16a', 'em16f', 'emc15', 'lea02', 'nzl10', 
              'res12', 'sar16', 'sea07', 'soa10', 'twn15', 'usa08'])
modell = set(['aus12', 'ca10h', 'ca10r', 'cub03', 'eu13a', 'eu13f',
              'eu13s', 'ear16', 'em16a', 'em16f', 'emc15', 'lea02', 'nzl10', 
              'res12', 'sar16', 'soa10', 'twn15'])
#modell = set(['cub03'])

## Read data for the different models

In [ ]:
modd = {}
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname))
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print (fname, end=' ')
        mod, point, info, sidx = model.load(fname)
        print (len(mod), end=' ')
        print ('done')
        modd[slist[0]] = {'model': mod, 'info': info}

In [ ]:
#
# find unique names for sources
unique_names = {}
unique_ids = {}
for key in ['eu13a', 'eu13f', 'eu13s', 'em16a', 'em16f']:
    names = set([])
    ids = {}
    for i, src in enumerate(modd[key]['model']): 
        names = names | set([src.name])
        if src.name in ids:
            ids[src.name].append(i)
        else:
            ids[src.name] = [i]
    unique_names[key] = names
    unique_ids[key] = ids
#
# 
bin_width = 0.1
for key in ['eu13a', 'eu13f', 'eu13s', 'em16a', 'em16f']:
    names = set([])
    ids = {}
    sources = []
    for name in unique_names[key]:
        tocc = 0. 
        for i, ids in enumerate(unique_ids[key][name]):
            #
            # this is for checking purpouses
            occ = np.array(modd[key]['model'][ids].mfd.get_annual_occurrence_rates())
            ss = sum(occ[:,1]) 
            tocc += ss
            #
            # stacking the MFDs
            if i == 0:
                nmfd = EEvenlyDiscretizedMFD.from_mfd(modd[key]['model'][ids].mfd)
            else:
                tmfd = modd[key]['model'][ids].mfd
                if isinstance(tmfd, TruncatedGRMFD):
                    tmfd = get_evenlyDiscretizedMFD_from_truncatedGRMFD(tmfd, nmfd.bin_width)
                nmfd.stack(tmfd)
        # 
        # check that initial and final rates are the same
        if abs(sum(np.array(nmfd.get_annual_occurrence_rates())[:,1]) - tocc) > 1e-10:
            print ('warning: total rates do not match with sum of individual ones')
        #
        # creating the new source
        src = copy.deepcopy(modd[key]['model'][ids])
        src.mfd = nmfd
        sources.append(src)
    modd[key]['model'] = sources
    print (key, len(sources))

## Plotting

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)

mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

bins = np.logspace(-10, -2, 17)

print (bins)

cnt = 1
lbls = []
xs = []
his = {}
mmint = 5.5
for key in sorted(modd.keys()):
    # Check if the model contains area sources
    if 'AreaSource' in modd[key]['info']['srcs_mmax']:

        print ('')
        print (key, modd[key]['info']['srcs_mmax']['AreaSource'])
        print (set(modd[key]['info']['trt_mmax']))

        fltr = 'Stable Continental Crust|Stable Continental Region|Stable Shallow Crust|Cratonic|Non_cratonic'; sxx = 0.25
        #fltr = 'Active Shallow Crust'; sxx = 0.35

        dend = get_rates_density(modd[key]['model'], mmint=mmint, trt=fltr)
        print ('Total number of sources {:d}'.format(len(modd[key]['model']))) 
        print ('Number of sources {:d}'.format(len(dend)))

        dens = []

        for _, value in dend.items():
            dens.append(value)

        if len(dens):
            print (min(dens), max(dens))
            his, _ = np.histogram(dens, bins)
            print ('Histogram sum {:d}'.format(sum(his)))

            imin = np.min(np.nonzero(his>0.))
            imax = np.max(np.nonzero(his>0.))

            lbls.append(key)
            xs.append(cnt)
            plt.plot([cnt, cnt], [min(dens), max(dens)],
                     linewidth=5, color='grey', alpha=.5)
        
            # plotting histogram
            dlt = 1
            nrm = his[imin:imax+dlt]/float(sum(his[imin:imax+dlt]))

            plt.barh(bins[imin:imax+dlt], 
                     nrm, 
                     height=np.diff(bins[imin:imax+2])*.85, 
                     left=cnt+0.1, 
                     edgecolor='black',
                     align='edge')
            smm = 0
            for y, x, h in zip(bins[imin:imax+dlt], nrm, his[imin:imax+dlt]):
                plt.text(cnt-sxx, y, '{0:>5d}'.format(h))
                smm += h
                print (smm, end=' ')
            print ('')

            # updating counter
            cnt += 1

ax.annotate('Min magnitude: {:.2f}'.format(mmint),
            xycoords="axes fraction",
            xy=(0.02, 0.97))

plt.yscale('log')
plt.ylabel('Eqks rate [1/[km2*yr]]', fontsize=14)
plt.grid(linestyle='--')
plt.ylim([1e-10, 1e-2])
aa = plt.xticks(xs, lbls, rotation='vertical')